In [1]:
import os
import pandas as pd
import re
import numpy as np
from pylab import rcParams
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
# Hash map de artistas
artistas = {}
plays = {}

file_list = []
playlists_in_query_list = []
# Listar todos os arquivos nos diretórios 'parquets/*/queries/*/pre_proc_tracks*
for files in os.walk('parquets'):
    for file in files[2]:
        if 'pre_proc_tracks' in file:
            file_list.append(files[0] + '/' + file)
        if 'playlists.parquet' in file:
            playlists_in_query_list.append(files[0] + '/' + file)

# Concatenar todos os parquets de playlists em queries em um único dataframe
# a coluna 'id' é o index
playlists = pd.concat([pd.read_parquet(file) for file in playlists_in_query_list], ignore_index=True)
playlists.set_index('id', inplace=True)

j = 0
# Para cada arquivo, abrir um parquet como pandas
for file in file_list:
    j += 1
    if j > 300:
        break
    df = pd.read_parquet(file)
    # O id da playlist é retirado pelo Regex 'pre_proc_tracks_(\d+).parquet'
    regex = 'pre_proc_tracks_(.+).parquet'
    id_playlist = re.search(regex, file).group(1)
    plays[id_playlist] = []
    # Em uma lista de diciionários, cada dicionário é um artista, com id e nome
    if 'artistas' not in df.columns:
        continue
    i = 0
    for track_artists in df['artistas']:
        i += 1
        # if (i > 100):
        #     break
        for artist in track_artists:
            if artist == None:
                continue
            if artist['id'] not in artistas:
                artistas[artist['id']] = artist['nome']
            if artist['id'] not in plays[id_playlist]:
                plays[id_playlist].append(artist['id'])

In [3]:
print('Quantidade de artistas:', len(artistas))
print('Quantidade de playlists:', len(plays))

Quantidade de artistas: 4601
Quantidade de playlists: 300


In [4]:
# # Lista de IDs únicos em artistas['id']
# unique_ids = artistas.keys()
# 
# # Cria uma matriz de adjacências completa de 0s
# matrix = pd.DataFrame(0, index=unique_ids, columns=unique_ids)


In [5]:
# for play in plays:
#     print('Computando playlist', play, '...')
#     # Cria uma lista de todos os artistas únicos
#     artists = plays[play]
# 
#     # Se não tem ao menos 20 artistas, ignora
#     if len(artists) < 20:
#         continue
# 
#     for artist in artists:
#         for artist2 in artists:
#             if artist != artist2:
#                 matrix.at[artist, artist2] += 1

In [18]:
G = nx.Graph()

# Cria nós do tipo playlist
for play in plays:
    # Nós do tipo playlist tem o nome da playlist e o dono como atributos
    name = playlists.loc[play]['name'] if play in playlists.index else 'Playlist ' + play
    owner = playlists.loc[play]['owner'] if play in playlists.index else 'Unknown'
    if play not in G.nodes:
        # G.add_node(play, name=name, owner=owner, type='playlist')
        G.add_node(play, type='playlist')

# Cria nós do tipo artista e arestas entre playlists e artistas
for play in plays:
    artists = plays[play]
    for artist in artists:
        # Nós do tipo artista tem o nome do artista como atributo
        if artist not in G.nodes:
            # G.add_node(artist, name=name, type='artist')
            G.add_node(artist, type='artist')
            G.add_edge(play, artist, type='playlist_artist')

# Cria arestas entre playlists se compartilham artistas
for play in plays:
    for play2 in plays:
        if play != play2:
            artists = plays[play]
            artists2 = plays[play2]
            common_artists = set(artists).intersection(set(artists2))
            G.add_edge(play, play2, type='playlist_playlist')

artists_to_remove = [artist for artist in G.nodes if G.nodes[artist]['type'] == 'artist' and len(list(G.neighbors(artist))) > 1]
G.remove_nodes_from(artists_to_remove)

playlists_to_remove = [play for play in G.nodes if G.nodes[play]['type'] == 'playlist' and len(list(G.neighbors(play))) == 0]
G.remove_nodes_from(playlists_to_remove)

# Arestas
edges = G.edges

# Exportart para Gephi
nx.write_gexf(G, 'artists_at_oneonly_one_play.gexf')

In [7]:
# # Tamanho da figura
# rcParams['figure.figsize'] = 20, 20
# 
# # Posicionamento dos nós
# pos = nx.spring_layout(G, scale=5000*G.order(), k=5/np.sqrt(G.order()), dim=2)
# 
# # Desenhar nós com o tamanho dos nós de playlist de acordo com o grau
# node_sizes = [G.degree(node) for node in G.nodes]
# nx.draw_networkx_nodes(G, pos, nodelist=playlist_nodes, node_color='blue' , node_size=node_sizes, alpha=0.5)
# nx.draw_networkx_nodes(G, pos, nodelist=artist_nodes, node_color='red', node_size=2)
# 
# # Desenhar arestas
# nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color='black', width=0.1)
# 
# plt.show()

In [8]:

# rcParams['figure.figsize'] = 14, 10
# pos = nx.spring_layout(G, scale=100000*G.order(), k=10/np.sqrt(G.order()))
# d = dict(G.degree)
# options = {
#     "node_color": "#A0CBE2",
#     "width": 0.1,
#     "edge_cmap": plt.cm.Blues,
#     "with_labels": False,
#     "nodelist": d, 
#     "node_size": [d[k] for k in d]
# }
# nx.draw(G, pos, **options)